In [18]:
import pandas as pd
import os
import shutil
import datetime as dt
import sys

try :
    list_poz = sorted(os.listdir('Relev_poz/'))
except:
    list_poz = ['!101']
    try:
        os.mkdir('Relev_poz/')
    except:
        pass
    os.system('say " Il nia pas de répertoire validee. Arrêt du programme. A demain."')
    sys.exit("Il n'y a pas de nouveau relevé de position valable. Arrêt du programme. A demain. \n \n \n ;) ")

for file in list_poz:
    if file[:9] != 'RelevPoz_':
        list_poz = ['!101']
        
if list_poz == ['!101']:
    os.system('say " Il nia pas de nouveau relevé de position valable. Arrêt du programme. A demain."')
    sys.exit("Il n'y a pas de répertoire validee. Arrêt du programme. A demain. \n \n \n ;) ")


try:
    os.mkdir('Relev_poz/Traité/')
except:
    pass

try:
    os.mkdir('Relev_poz/Doublons/')
except:
    pass
    
RAB = []
HORO = []
BOO = []
CLEAN = []
for foo,bar in enumerate(list_poz):
    if list_poz[foo][:8] == 'RelevPoz':
        rab = bar[9:19].replace('_','')[4:]+bar[9:19].replace('_','')[2:4]+bar[9:19].replace('_','')[:2]
        horo = bar[20:].replace('_','')
        RAB.append(rab)
        HORO.append(horo)
for i in set(RAB):
    TEMPHORO = []
    for j,k in enumerate(RAB):
        if i==k:
            TEMPHORO.append('RelevPoz_'+k[6:10]+'_'+k[4:6]+'_'+k[:4]+'_'+HORO[j][:2]+'_'+HORO[j][2:4]+'_'+HORO[j][4:])
    CLEAN.append(max(TEMPHORO))

CLOSE = []
ERROR = []

def get_creation_date(file):
    stat = os.stat(file)
    try:
        
        return stat.st_birthtime
    except AttributeError:
        return stat.st_mtime

try:
    LaBase = pd.read_csv('LaBase_adj.csv')
    try:
        LaBase = LaBase.drop('Unnamed: 0',axis=1)
    except:
        pass
    
        """
    Pour remettre toutes dates de LaBase au même format

    LaBase['Date'] = LaBase.Date.apply(
        lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
    """   

    
except:
    pass
LaBase=LaBase.drop_duplicates(subset=(['Date','Symbol']),keep='last')
try:
    megadf = pd.read_csv('Relev_poz/Traité/megadf.csv')
    try:
        megadf = megadf.drop('Unnamed: 0',axis=1)
    except:
        pass
except:
    megadf = pd.DataFrame()
try:
    globaldf = pd.read_csv('Relev_poz/Traité/globaldf.csv')
    try:
        globaldf = globaldf.drop('Unnamed: 0',axis=1)
    except:
        pass
except:
    globaldf = pd.DataFrame()
try:
    tickerdf = pd.read_csv('Relev_poz/Traité/tickerdf.csv')
    try:
        tickerdf = tickerdf.drop('Unnamed: 0',axis=1)
    except:
        pass
except:
    tickerdf = pd.DataFrame()

    
for foo,bar in enumerate(CLEAN):
    df = (pd.read_csv('Relev_poz/'+bar))
    try:
        df = df.drop('Unnamed: 0',axis=1)
    except:
        pass
     
    creation_date = dt.datetime.fromtimestamp(get_creation_date(file='Relev_poz/'+bar))
    ddate = creation_date
    heure = str(creation_date)
    jour = creation_date.weekday()
    if jour == 6 and heure > '05:00:00' and heure < '23:50:00':
        yesterday = str(ddate-dt.timedelta(2))[:5]+str(ddate-dt.timedelta(2))[5:7]+str(ddate-dt.timedelta(2))[7:10]

    elif jour == 0 and heure > '05:00:00' and heure < '21:50:00':
        yesterday = str(ddate-dt.timedelta(3))[:5]+str(ddate-dt.timedelta(3))[5:7]+str(ddate-dt.timedelta(3))[7:10]
        
    elif jour == 1 and heure > '05:00:00' and heure < '21:50:00':
        yesterday = str(ddate-dt.timedelta(1))[:5]+str(ddate-dt.timedelta(1))[5:7]+str(ddate-dt.timedelta(1))[7:10]        

    elif jour == 2 and heure > '05:00:00' and heure < '21:50:00':
        yesterday = str(ddate-dt.timedelta(1))[:5]+str(ddate-dt.timedelta(1))[5:7]+str(ddate-dt.timedelta(1))[7:10]

    elif jour == 3 and heure > '05:00:00' and heure < '21:50:00':
        yesterday = str(ddate-dt.timedelta(1))[:5]+str(ddate-dt.timedelta(1))[5:7]+str(ddate-dt.timedelta(1))[7:10]

    elif jour == 4 and heure > '05:00:00' and heure < '21:50:00':
        yesterday = str(ddate-dt.timedelta(1))[:5]+str(ddate-dt.timedelta(1))[5:7]+str(ddate-dt.timedelta(1))[7:10]
    else:
        os.system('say "Nous ne sommes pas dans un créneau où il est pertient dopérer"')
        os.exit("Nous ne sommes pas dans un créneau où il est pertient d'opérer")

    
    for titi in range(0,len(df)):
        try:
            CLOSE.append(round(float(LaBase[(LaBase['Date']==yesterday)&(LaBase['Symbol']==df.iloc[titi]['Ticker'])].Close),2))
        except:
            CLOSE.append('NA')
            ERROR.append((yesterday,df.iloc[titi]['Ticker']))
            print('Il y a ',len(ERROR),' erreurs pour les tupples ', ERROR)
            print(' /!\ Attention! On retire les lignes NA! /!\ ')
    
    df['Average Cost'] = round(df['Average Cost'],2)
    df['Date du Relevé'] = bar[15:19]+'-'+bar[12:14]+'-'+bar[9:11]
    df['Exposition'] = round(df['Quantité'] * df['Average Cost'],2)
    df['Date'] = yesterday
    df['Close'] = CLOSE
    df = df[df['Close'] != 'NA']
    df['Latent PnL'] = (df['Close'] - df['Average Cost']) * df['Quantité']
    df = df[['Date du Relevé','Compte','Type','Ticker','Quantité','Average Cost','Exposition','Date','Close','Latent PnL']]
    CLOSE = []
    megadf = pd.concat((megadf,df), ignore_index=True)


    
df1 = pd.DataFrame()
for foo,bar in enumerate(megadf['Ticker'].unique()):
    df = megadf[megadf['Ticker'] == bar]
    df1['Date'] = pd.DataFrame([df.iloc[-1]['Date']], columns=['Date'])
    df1['Compte'] = pd.DataFrame([df.iloc[-1]['Compte']], columns=['Compte'])
    df1['Ticker'] = pd.DataFrame([bar], columns=['Ticker'])
    df1['Type'] = pd.DataFrame([df.iloc[-1]['Type']], columns=['Type'])
    df1['Quantité'] = pd.DataFrame([df['Quantité'].sum()],columns=['Quantité'])
    df1['Exposition'] = pd.DataFrame([df['Exposition'].sum()],columns=['Exposition'])
    df1['Latent PnL'] = pd.DataFrame([df['Latent PnL'].sum()],columns=['Latent PnL'])

    tickerdf = pd.concat((tickerdf,df1),ignore_index=True)

df = pd.DataFrame()
for bar in sorted(set(tickerdf['Date'])):
    df['Date'] = pd.DataFrame([bar],columns=['Date'])
    df['Nbre Positions'] = pd.DataFrame([tickerdf[tickerdf['Date']==bar].shape[0]],columns=['Nbre Positions'])
    df['Nbre Tickers'] = pd.DataFrame([tickerdf[(tickerdf['Date']==bar) & (tickerdf['Ticker'])]['Ticker'].unique().shape[0]],columns=['Nbre Tickers'])
    df['Nbre Long'] = pd.DataFrame([tickerdf[(tickerdf['Date']==bar)&(tickerdf['Quantité']>0)].shape[0]],columns=['Nbre Long'])
    df['Nbre Short'] = pd.DataFrame([tickerdf[(tickerdf['Date']==bar)&(tickerdf['Quantité']<0)].shape[0]],columns=['Nbre Short'])
    df['Exposition Long'] = pd.DataFrame([tickerdf[(tickerdf['Date']==bar)&(tickerdf['Quantité']>0)]['Exposition'].sum()],columns=['Exposition Long'])
    df['Exposition Short'] = pd.DataFrame([tickerdf[(tickerdf['Date']==bar)&(tickerdf['Quantité']<0)]['Exposition'].sum()],columns=['Exposition Short'])
    df['Exposition Totale'] = pd.DataFrame([tickerdf[(tickerdf['Date']==bar)&(tickerdf['Quantité'])]['Exposition'].sum()],columns=['Exposition Totale'])
    df['Latent PnL'] = pd.DataFrame([tickerdf[(tickerdf['Date']==bar)]['Latent PnL'].sum()],columns=['Latent PnL'])
    globaldf = pd.concat((globaldf,df),ignore_index=True)
    
megadf.to_csv('Relev_poz/Traité/megadf.csv')
tickerdf.to_csv('Relev_poz/Traité/tickerdf.csv')
globaldf.to_csv('Relev_poz/Traité/globaldf.csv')

for file in CLEAN:
    shutil.move('Relev_poz/'+file,'Relev_poz/Traité/'+file)
list_poz = sorted(os.listdir('Relev_poz/'))
for file in list_poz:
    if file[:9] == 'RelevPoz_':
        shutil.move('Relev_poz/'+file,'Relev_poz/Doublons/'+file)

SystemExit: Il n'y a pas de répertoire validee. Arrêt du programme. A demain. 
 
 
 ;) 

In [ ]:
megadf.duplicated().sum()

In [ ]:
megadf.tail()

In [ ]:
tickerdf.sort_values(by='Date')

In [ ]:
tickerdf.sort_values(by='Latent PnL',ascending=False)

In [ ]:
tickerdf

In [ ]:
globaldf